# Sequential 模型

### 引入

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 何时使用Sequential模型

Sequential模型类似一个 **层的堆栈**，其中每个层正好**只有一个输入张量和一个输出张量**。

如下所示，如何使用Sequential模型。

In [ ]:
# 定义一个具有三个层的Sequential模型
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# 使用tf.ones创建一个初值为1的3*3张量，并作为输入调用模型
x = tf.ones((3, 3))
y = model(x)

print(y.shape)

上面的代码等效于如下方法：

In [ ]:
# 首先创建3个层
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# 同样的，使用tf.ones创建一个初值为1的3*3张量，通过嵌套调用达到上述效果
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

Sequential模型**不适用**与以下场景：


+   模型需要多个输入或输出
+   模型中任何一个层需要多个输入或输出
+   需要共享某些层的权重时(共享权重)
+   需要非线性拓扑构建模型时(如，残差连接，多分支模型)

### 创建一个Sequential模型

你可以将一个 **层的列表**传递给Sequential的构造函数，从而构建模型：

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

创建完成之后，**层**作为Sequential模型的一个属性，可以通过`layers`进行访问：

In [ ]:
print(dir(model))  # 查看模型的属性或方法

model.layers

你还可以使用`add()`方法，通过逐个添加层的方式来创建Sequential模型

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

值得注意的是，你还可以相应使用`pop()`方法删除层：Sequential模型的行为非常类似于层堆栈，后进先出。

In [ ]:
model.pop()
print(len(model.layers))  # 2

同样值得注意的是，和Keras中任何层或者模型一样，Sequential的构造函数也接受一个`name`参数。当你使用TensorBoard的时候，有一个言简意赅的标识是非常有用的。

In [ ]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

### 预先指定输入形状

通常情况下，Keras中所有的层需要知道输入形状，以此来创建相对应的权重变量。所以，当你按照如下方式创建层时，它是没有创建权重的。

In [ ]:
layer = layers.Dense(3)
layer.weights  # Empty

上面的层在首次调用时，会根据输入创建一个权重，因此权重的形状是依据输入的形状决定的。

In [ ]:
# 用一个测试输入调用层
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # 现在该层有一个权重，且形状为 (4, 3)和(3,)
#layer.weights是一个列表，就是训练所得的参数（W和b）

当然，上述权重的创建也同样适用于Sequential模型，当你实例化模型且没有预先指定输入形状，模型是没有被“创建”的：也就意味着没有对应的权重(当调用`model.weights`时会报错)，当输入数据时，会根据输入形状创建权重。

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # 到这一步时，权重没有被创建

# 也就是说，你不可以调用如下代码：
# model.weights

# 当然，你也不可以调用如下代码：
# model.summary()

# 使用一个测试输入调用模型
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6，每层2个

当模型“创建”后，你可以调用`summary()`方法，查看模型的相关信息：

In [ ]:
model.summary()

然而，如果我们能在渐进式的构建模型时，调用`summary()`查看相关信息(包括输入形状等)将会非常有帮助。因此，你可以通过添加`Input`对象的方式，指定输入的形状（形状中不包括批维）。

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

model.summary()

由上面信息我们可以发现，`Input`对象不是`model.layers`的一部分，所以它不是层：

In [ ]:
model.layers

还有一种更简单的方式指定输入形状，即在模型的第一层中，传入`input_shape`参数

In [ ]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

通过这种方式，在不需要传入输入数据的情况下，模型将会根据预定义的输入形状进行创建，且模型具有对应的权重以及输入形状。

通常情况下，如果你预先知道模型的输入形状的话，最佳的做法就是如上指定输入形状。

### 通用的调试流程：add() + summary()

当你在构建一个新的Sequential模型结构时，通过`add()`方法渐进式的添加层以及频繁的使用`summary()`查看相关信息将会非常有帮助。如下示例，通过上述方式，将可以查看由`Conv2D`和`MaxPooling2D`层构建的模型，是如何对图片进行降采样得到feature maps(**在cnn的每个卷积层，数据都是以三维形式存在的，你可以把它看成许多个二维图片叠在一起，其中每一个层为一个feature map**)

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB 图像
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# 你可以猜到此时的输出形状是什么么？可能猜不出
# 将信息打印出来:
model.summary()

# 答案是: (40, 40, 32), 所以，我们可以继续进行降采样...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# 现在呢?
model.summary()

# 现在我们得到 4x4 feature maps, 此时应用最大池化.
model.add(layers.GlobalMaxPooling2D())

# 最后, 我们添加一个分类层
model.add(layers.Dense(10))

很实用，对吧？

### 你可以用创建的模型干什么？
当你完成了模型结构的构建，可能你想要用它来做如下事情：

+ 训练、评估、推断你的模型
+ 将你的模型打包保存在硬盘中
+ 利用多个GPU加速模型的训练

### 使用顺序模型进行特征提取

一旦创建好的Sequential模型，它的行为就类似于函数式API模型。这也就意味着每一层拥有着输入和输出属性。这些属性可以简化一些工作，如通过提取模型的所有中间层，可以快速的创建一个Sequential模型。

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers]  # 每层的输出
)

# 使用测试输入调用特征提取
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
print(len(features))
print(features[0].shape)  # 第一个卷积层抽取的特征

下面是类似的例子，对单个层进行特征提取：

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),  # 通过名字引用
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)

# 使用测试输入调用特征提取
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
print(features.shape)

### 使用Sequential模型进行迁移学习

迁移学习冻结模型底部的层，仅仅使用顶部的层进行训练，如果你不熟悉迁移学习，请先看[迁移学习指南](https://tensorflow.google.cn/guide/keras/transfer_learning/)

这里将使用Sequential模型，举例两个常见的迁移学习策略。

首先，我们需要有一个Sequential模型，然后你需要冻结除了最后一层之外的其他层，在这个例子中，你只需要通过迭代`model.layers`，对除了最后一层之外的每个层设置`layer.trainable = False`，如下所示：

In [ ]:
# 创建模型结构
model = keras.Sequential([
    keras.Input(shape=(784))
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# 想必你可能想要先加载预训练权重
model.load_weights(...)

# 冻结除了最后一层之外的其他层
for layer in model.layers[:-1]:
  layer.trainable = False

# 编译和训练 (只会更新最后一层的权重)
model.compile(...)
model.fit(...)

基于同样的构建策略，我们使用Sequential将一个预训练的模型和一个新初始化的分类层堆叠在一起，如下所示：

In [ ]:
# 加载一个带有预训练权重的卷积基
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# 冻结该模型
base_model.trainable = False

# 使用Sequential模型，将一个分类器添加到顶部
model = keras.Sequential([
    base_model,
    layers.Dense(1000),
])

# 编译 & 训练
model.compile(...)
model.fit(...)

如果你进行迁移学习，你可能会经常使用这两种模式。

以上就是需要了解的Sequential模型的全部内容！ 
要了解有关在Keras中构建模型的更多信息，请参见：
+ [函数式API指南](https://tensorflow.google.cn/guide/keras/functional/)
+ [通过子类创建新的层和模型指南](https://tensorflow.google.cn/guide/keras/custom_layers_and_models/)